In [22]:
# import pandas as pd

# # Load the dataset
# df = pd.read_excel('data/Adidas US Sales Datasets.xlsx')

# # Display basic information about the dataset
# print(df.info())

# # Display the first few rows to understand the data
# print(df.head())

# file_path = 'data/Adidas US Sales Datasets.xlsx'

# # Load the Excel file and check the sheet names
# xls = pd.ExcelFile(file_path)
# xls.sheet_names

# # Clean the dataset by skipping the initial empty rows and using the correct headers
# df = pd.read_excel(file_path, sheet_name='Data Sales Adidas', skiprows=4)

# # Display the cleaned data to confirm the structure
# df.head()

# df=df.drop(columns=['Unnamed: 0','Retailer ID'])

# # Save the cleaned dataset to a CSV file
# output_csv_path = 'data/Adidas_US_Sales_Cleaned.csv'
# df.to_csv(output_csv_path, index=False)

# output_csv_path



## Application

In [25]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import joblib

# Load the user's uploaded CSV file
file_path = 'data/Adidas_US_Sales_Cleaned.csv'
data = pd.read_csv(file_path)



# Selecting relevant features for the model (dropping non-numeric and categorical data)
# We'll predict 'Total Sales' based on 'Price per Unit', 'Units Sold', and 'Operating Profit'
X = data[['Price per Unit', 'Units Sold', 'Operating Profit']]
y = data['Total Sales']

# Splitting the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Creating and training the Linear Regression model
model = LinearRegression()
model.fit(X_train, y_train)

# Making predictions on the test data
y_pred = model.predict(X_test)

# Save the trained model to a file using joblib
model_file_path = 'data/linear_regression_model.pkl'
joblib.dump(model, model_file_path)

# Calculating the model's performance using Mean Squared Error (MSE)
mse = mean_squared_error(y_test, y_pred)

# Coefficients of the model
coefficients = model.coef_

# Displaying the results
mse, coefficients

(np.float64(1253241097.91393),
 array([991.90177354, 221.33278234,   1.62362108]))

In [8]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import joblib
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import optuna
import warnings
warnings.filterwarnings("ignore")
# Step 1: Load the dataset
data = pd.read_csv('data/Adidas_US_Sales_Cleaned.csv')

# Step 2: Convert 'Invoice Date' to datetime and extract year and month
data['Invoice Date'] = pd.to_datetime(data['Invoice Date'])
data['year'] = data['Invoice Date'].dt.year
data['month'] = data['Invoice Date'].dt.month

# Step 3: Drop unnecessary columns
data = data.drop(['Invoice Date', 'Operating Profit', 'Operating Margin'], axis=1)

# Step 4: One-hot encoding for categorical variables
data = pd.get_dummies(data, columns=['Retailer', 'Region', 'State', 'City', 'Product', 'Sales Method'], drop_first=True)

# Step 5: Define the features and target
X = data.drop(['Units Sold'], axis=1)  # Features
y = data['Units Sold']  # Target variable

# Step 6: Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Step 7: Scale the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Save the scaler and columns
joblib.dump(scaler, 'scaler.joblib')
joblib.dump(X.columns, 'columns.pkl')

# Step 8: Optimize with Optuna
# Objective function for Optuna optimization
def objective(trial):
    # Define hyperparameters to tune
    fit_intercept = trial.suggest_categorical('fit_intercept', [True, False])

    # Train the model with the selected hyperparameters
    model = LinearRegression(fit_intercept=fit_intercept)
    model.fit(X_train_scaled, y_train)

    # Evaluate the model on the test set
    y_pred = model.predict(X_test_scaled)
    rmse = mean_squared_error(y_test, y_pred, squared=False)
    
    return rmse

# Run Optuna optimization
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=50)

# Best hyperparameters
best_params = study.best_params
print(f"Best Hyperparameters: {best_params}")

# Train the final model using the best hyperparameters
lr_model = LinearRegression(**best_params)
lr_model.fit(X_train_scaled, y_train)

# Step 9: Save the trained model
joblib.dump(lr_model, 'linear_regression_sales_model_optuna.joblib')

# Step 10: Predict for maximum date and compare actual vs predicted

# Load the trained model, scaler, and columns
lr_model = joblib.load('linear_regression_sales_model_optuna.joblib')
scaler = joblib.load('scaler.joblib')
original_columns = joblib.load('columns.pkl')

# Find the max date in the dataset
max_date = data['year'].max(), data['month'].max()

# Extract features for the max date
test_data = data[(data['year'] == max_date[0]) & (data['month'] == max_date[1])].drop('Units Sold', axis=1)

# Scale the input data
input_scaled = scaler.transform(test_data)

# Predict sales
predicted_qty = lr_model.predict(input_scaled)

# Compare actual and predicted values
actual_qty = y[(data['year'] == max_date[0]) & (data['month'] == max_date[1])]
comparison_df = pd.DataFrame({
    'Actual Sales': actual_qty,
    'Predicted Sales': predicted_qty
})

print(comparison_df)


[I 2024-10-09 14:58:52,812] A new study created in memory with name: no-name-5efd9fc2-e98f-42ba-8148-ec9ac39b65d3
[I 2024-10-09 14:58:52,829] Trial 0 finished with value: 59.3676250059195 and parameters: {'fit_intercept': True}. Best is trial 0 with value: 59.3676250059195.
[I 2024-10-09 14:58:52,844] Trial 1 finished with value: 264.41248400238936 and parameters: {'fit_intercept': False}. Best is trial 0 with value: 59.3676250059195.
[I 2024-10-09 14:58:52,860] Trial 2 finished with value: 59.3676250059195 and parameters: {'fit_intercept': True}. Best is trial 0 with value: 59.3676250059195.
[I 2024-10-09 14:58:52,875] Trial 3 finished with value: 264.41248400238936 and parameters: {'fit_intercept': False}. Best is trial 0 with value: 59.3676250059195.
[I 2024-10-09 14:58:52,889] Trial 4 finished with value: 59.3676250059195 and parameters: {'fit_intercept': True}. Best is trial 0 with value: 59.3676250059195.
[I 2024-10-09 14:58:52,903] Trial 5 finished with value: 59.3676250059195 a

Best Hyperparameters: {'fit_intercept': True}
      Actual Sales  Predicted Sales
468            200       181.970437
469            375       383.317586
470            525       554.792205
471            300       302.362774
472            225       223.974202
...            ...              ...
9634           128       125.722401
9635           128       114.972401
9636           116        84.722401
9637           123        76.222401
9638           144        99.972401

[667 rows x 2 columns]


## Presentation Code

In [7]:
# Importing necessary libraries
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import joblib
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import optuna
import warnings
warnings.filterwarnings("ignore")

# Step 1: Load the dataset
data = pd.read_csv('data/Adidas_US_Sales_Cleaned.csv')

# Step 2: Convert 'Invoice Date' to datetime and extract year and month
data['Invoice Date'] = pd.to_datetime(data['Invoice Date'])
data['year'] = data['Invoice Date'].dt.year
data['month'] = data['Invoice Date'].dt.month

# Step 3: Drop unnecessary columns
data = data.drop(['Invoice Date', 'Operating Profit', 'Operating Margin'], axis=1)

# Step 4: One-hot encoding for categorical variables
data = pd.get_dummies(data, columns=['Retailer', 'Region', 'State', 'City', 'Product', 'Sales Method'], drop_first=True)

# Step 5: Define the features and target
X = data.drop(['Units Sold'], axis=1)  # Features
y = data['Units Sold']  # Target variable

# Step 6: Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Step 7: Scale the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Save the scaler and columns
joblib.dump(scaler, 'scaler.joblib')
joblib.dump(X.columns, 'columns.pkl')

# Step 8: Optimize with Optuna
# Objective function for Optuna optimization
def objective(trial):
    # Define hyperparameters to tune
    fit_intercept = trial.suggest_categorical('fit_intercept', [True, False])

    # Train the model with the selected hyperparameters
    model = LinearRegression(fit_intercept=fit_intercept)
    model.fit(X_train_scaled, y_train)

    # Evaluate the model on the test set
    y_pred = model.predict(X_test_scaled)
    rmse = mean_squared_error(y_test, y_pred, squared=False)
    
    return rmse

# Run Optuna optimization
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=50)

# Best hyperparameters
best_params = study.best_params
print(f"Best Hyperparameters: {best_params}")

# Train the final model using the best hyperparameters
lr_model = LinearRegression(**best_params)
lr_model.fit(X_train_scaled, y_train)

# Step 9: Save the trained model
joblib.dump(lr_model, 'linear_regression_sales_model_optuna.joblib')

# Step 10: Predict for maximum date and compare actual vs predicted

# Load the trained model, scaler, and columns
lr_model = joblib.load('linear_regression_sales_model_optuna.joblib')
scaler = joblib.load('scaler.joblib')
original_columns = joblib.load('columns.pkl')

# Find the max date in the dataset
max_date = data['year'].max(), data['month'].max()

# Extract features for the max date
test_data = data[(data['year'] == max_date[0]) & (data['month'] == max_date[1])].drop('Units Sold', axis=1)

# Scale the input data
input_scaled = scaler.transform(test_data)

# Predict sales
predicted_qty = lr_model.predict(input_scaled)

# Compare actual and predicted values
actual_qty = y[(data['year'] == max_date[0]) & (data['month'] == max_date[1])]
comparison_df = pd.DataFrame({
    'Actual Sales': actual_qty.values,  # Ensure alignment of values
    'Predicted Sales': predicted_qty
})

# Display the comparison dataframe
print(comparison_df)


ModuleNotFoundError: No module named 'pandas'